# Bokeh Interactive Visualizations

Bokeh is a Python library for creating interactive visualizations ranging from simple plots to complex dashboards.  Under the hood, Bokeh utilizes JavaScript, but in order to use Bokeh you don't need to write any JavaScript yourself.

In this tutorial we are going to visualize the leverage effect in SPY using Bokeh.

### Importing Packages

Let's begin by importing packages.

In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr

### Reading-In Stock Data into a `DataFrame`

Next, let's read-in some price data from Yahoo Finance.  

In [2]:
# reading in data
df_spy = pdr.get_data_yahoo('SPY', start='2016-01-01', end='2021-07-31')

# cleaning the DataFrame
df_spy = df_spy.round(2)
df_spy.reset_index(inplace=True)
df_spy.columns = df_spy.columns.str.lower().str.replace(' ','_')
lst_cols = ['high', 'low', 'open', 'adj_close', 'volume',]
df_spy.drop(columns=lst_cols, inplace=True)

# adding returns and realized volatility
df_spy['ret'] = df_spy['close'] / df_spy['close'].shift(1) - 1
df_spy['realized_vol'] = df_spy['ret'].rolling(42).std() * np.sqrt(252)

df_spy.head()

,date,close,ret,realized_vol
0,2016-01-04,201.02,NaN,NaN
1,2016-01-05,201.36,0.001691,NaN
2,2016-01-06,198.82,-0.012614,NaN
3,2016-01-07,194.05,-0.023992,NaN
4,2016-01-08,191.92,-0.010977,NaN


### Single Plot - `close` Prices

Generating a single plot of `close` prices can be done with a small amount of code.  Notice that the default behavior is to include an interactive toolbar on the right of the graph that allows for actions like panning and zooming.

In [3]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

p = figure(width=600, height=250, x_axis_type='datetime')
p.line(x=df_spy['date'], y=df_spy['close'])
show(p)

Loading BokehJS ...

**Experiment Challenge:** Experiement with the interactive tools in the plot.

### Multiple Plots

We can graph all three series - `close`, `ret`, `realized_vol` - in a single plot by utilizing the `column()` function.

In [4]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column
output_notebook()

# defining multiple plots
p1 = figure(width=600, height=250, x_axis_type='datetime')
p1.line(x=df_spy['date'], y=df_spy['close'])

p2 = figure(width=600, height=250, x_axis_type='datetime')
p2.line(x=df_spy['date'], y=df_spy['ret'])

p3 = figure(width=600, height=250, x_axis_type='datetime')
p3.line(x=df_spy['date'], y=df_spy['realized_vol'])

# putting them together using column()
show(column(p1, p2, p3))

Loading BokehJS ...

**Experiment Challenge:** What do you notice when you perform an interactive action on one of the plots?

### Linked Zooming

As you could see from the previous graphs, the three graphs are not linked in that panning and zooming in one plot has no effect on the others.  In this next graph, we link the zooming of all three plots.  This is done using the `grid()` function.

In [5]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
output_notebook()

# defining plot options all at once
plot_options = dict(width=600, height=250, x_axis_type='datetime', tools='wheel_zoom, reset, box_zoom')

# defining multiple plots - notice the change in the x_range for p2 and p3
p1 = figure(**plot_options)
p1.line(x=df_spy['date'], y=df_spy['close'])

p2 = figure(x_range=p1.x_range,  **plot_options)
p2.line(x=df_spy['date'], y=df_spy['ret'])

p3 = figure(x_range=p1.x_range, **plot_options)
p3.line(x=df_spy['date'], y=df_spy['realized_vol'])

# putting all plots into a grid plot
p = gridplot([[p1], 
              [p2], 
              [p3]]
            )

show(p)

Loading BokehJS ...

### Linked Brushing

Brushing is an interesting way of highlighting sections of a graph.  Unfortunately, it doesn't work on line graphs, so let's use circle glyphs instead.  Here is a plot with linked brushing.

In [6]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
output_notebook()

# defining plot options all at once
plot_options = dict(width=600, height=250, x_axis_type='datetime', tools='wheel_zoom,reset,box_zoom,box_select')

# placing df_spy into a ColumnDataSource object
cds_spy = ColumnDataSource(df_spy)

# defining multiple plots
p1 = figure(**plot_options)
p1.circle(x='date', y='close', source=cds_spy)

p2 = figure(x_range=p1.x_range, **plot_options)
p2.circle(x='date', y='ret', source=cds_spy)

p3 = figure(x_range=p1.x_range, **plot_options)
p3.circle(x='date', y='realized_vol', source=cds_spy)

# putting all plots into a grid plot
p = gridplot([[p1], 
              [p2], 
              [p3]]
            )

show(p)

Loading BokehJS ...

**Code Challenge:** Modify the code above to use square glyphs instead of circles.

### Hover Tools

Hover tools are a way to make your plots more immediately readable.

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool
output_notebook()

# placing df_spy into a ColumnDataSource object
cds_spy = ColumnDataSource(df_spy)

# defining the lover tool
hover = HoverTool(
        tooltips=[
            ('date', '@date{%F}'),
            ('adj_close', '$@{close}{0.2f}'),
        ],
        formatters={'@date':'datetime',
                   '@close':'printf'
                   },
    )

# defining plot options all at once
plot_options = dict(width=600, height=250, x_axis_type='datetime', 
                    tools=[hover, 'wheel_zoom,reset,box_zoom'])

# defining multiple plots
p1 = figure(**plot_options)
p1.line(x='date', y='close', source=cds_spy)

p2 = figure(x_range=p1.x_range, **plot_options)
p2.line(x='date', y='ret', source=cds_spy)

p3 = figure(x_range=p1.x_range, **plot_options)
p3.line(x='date', y='realized_vol', source=cds_spy)

# putting all plots into a grid plot
p = gridplot([[p1], 
              [p2], 
              [p3]]
            )

show(p)

**Code Challenge:** During the pandemic, what was the lowest price of SPY and on what date?